## 실습#1 우리학교는 어디에?

### 1. 라이브러리 설정

In [1]:
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

###  2. 데이터 수집하기(셀레니움 활용)

In [2]:
#웹드라이버의 주소를 적고, 웹사이트 불러오기
driver = webdriver.Chrome('C:/Users/kogas/Downloads/chromedriver')
driver.get("http://krsearch.co.kr/")

In [3]:
# 학교 리스트
q_list = ['원곡고등학교','상록고등학교', '양지고등학교', '행신고등학교', '안산고등학교', '초지고등학교', '불곡고등학교']

In [4]:
# 고등학교 검색
all_data = []
for q in q_list:
    search_element = driver.find_element_by_name('p')
    search_element.clear()
    search_element.send_keys(q)
# 검색 후 클릭
    search_button = driver.find_element_by_class_name("searchbtn2")
    search_button.click()
# 검색결과 수집
    pageSource = driver.page_source
    bsObj = BeautifulSoup(pageSource, 'html.parser')
    school=[]
    data = bsObj.find_all('td',{"class":"bbslist"})
    school.append(data[0].get_text().strip())
    school.append(data[1].get_text().strip())
    driver.get("http://krsearch.co.kr/")
    all_data.append(school)

In [5]:
#데이터 확인
all_data

[['원곡고등학교', '경기도 안산시 단원구 원곡동 965-1번지'],
 ['상록고등학교', '경기도 안산시 상록구 본오동 산127-1번지'],
 ['양지고등학교', '경기도 안산시 단원구 고잔동 713번지'],
 ['행신고등학교', '경기도 고양시 덕양구 행신동 790번지'],
 ['안산고등학교', '경기도 안산시 상록구 청룡4길 8'],
 ['초지고등학교', '경기도 안산시 단원구 광덕1로 13'],
 ['불곡고등학교', '경기도 성남시 분당구 금곡로 149번지']]

In [6]:
# DataFrame 형태로 바꾸기
fdata = pd.DataFrame(all_data, columns = ['name','address'])

In [7]:
fdata

,name,address
0,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지
1,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지
2,양지고등학교,경기도 안산시 단원구 고잔동 713번지
3,행신고등학교,경기도 고양시 덕양구 행신동 790번지
4,안산고등학교,경기도 안산시 상록구 청룡4길 8
5,초지고등학교,경기도 안산시 단원구 광덕1로 13
6,불곡고등학교,경기도 성남시 분당구 금곡로 149번지


In [8]:
#데이터 csv 파일로 저장
fdata.to_csv('school.csv')

### 3. 위/경도 정보 만들기 

#### 3-1 방법#1. Geocode by Awesome Table 사용

In [33]:
#위,경도 데이터 전환 후 불러오기
school_df = pd.read_csv('./school2.csv')

In [34]:
school_df.head(5)

,name,address,Latitude,Longitude
0,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053
1,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971
2,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268
3,행신고등학교,경기도 고양시 덕양구 행신동 790번지,37.616131,126.842504
4,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672


#### 3-2 방법#2. google geocode API를 사용

In [24]:
school_df = pd.read_csv('./school.csv', engine = 'python')

In [28]:
school_df = school_df.drop('Unnamed: 0', axis=1)

In [29]:
school_df

,name,address
0,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지
1,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지
2,양지고등학교,경기도 안산시 단원구 고잔동 713번지
3,행신고등학교,경기도 고양시 덕양구 행신동 790번지
4,안산고등학교,경기도 안산시 상록구 청룡4길 8
5,초지고등학교,경기도 안산시 단원구 광덕1로 13
6,불곡고등학교,경기도 성남시 분당구 금곡로 149번지


#### API를 활용해 위경도로 바꾸는 함수 만들기

In [30]:
import requests
import json

def get_latitude_longitude(addr):
    '''
    DESCRIPTION: 주소를 입력하면 위도(latitude)/경도(longitude)를 반환하는 함수 
    ex) get_latitude_longitude('경기도 안산시 상록구 한양대학로 55')
    '''
    location = addr
    response = requests.get('http://maps.googleapis.com/maps/api/geocode/json?sensor=false&language=ko&address=' + location)    
    loc_dict = json.loads(response.text)
    latitude = loc_dict['results'][0]['geometry']['location']['lat'] 
    longitude = loc_dict['results'][0]['geometry']['location']['lng']
    return [latitude, longitude]

In [33]:
# 위/경도 정보로 저장해 새로운 데이터 형성 후 저장하기
latitude_list = []
longitude_list = []
for i in school_df['address'].values:
    lat, lon = get_latitude_longitude(i)
    latitude_list.append(lat)
    longitude_list.append(lon)

school_df['latitude'] = latitude_list
school_df['longitude'] = longitude_list

# 수정 17.10.10 - API이용 버전 위/경도 반영 DataFrame 저장
school_df.to_csv('./꿈의대학_고등학교 리스트.csv', index=False, encoding='euc-kr')

###  4. 지도로 표시하기

In [41]:
import folium
# 오류수정 17.10.10 - Latitude, Longitude를 latitude, longitude로 변경. 
hanyang_univ_map = folium.Map(location=[37.298524, 126.834712], zoom_start=10)
for l in school_df[['name', 'latitude', 'longitude']].values:
#    folium.Marker(l[1:],popup=l[0], icon=folium.Icon(color='red')).add_to(한양대지도)
    folium.CircleMarker(l[1:], popup=l[0], color='red', fill_color='red', radius=5).add_to(hanyang_univ_map)

In [42]:
hanyang_univ_map

## 실습#2 제일 가까운 학교는? 

###  1. 기준 위치 설정하기

In [43]:
# 주소를 기반으로 위/경도 데이터 얻기
#my_location = get_latitude_longitude('경기도 안산시 상록구 사3동 한양대학로55 한양대학교 경상대학')
my_location = [37.296755, 126.836194]
my_location_name = '한양대학교 제4공학관'

### 2. 위치 계산기 만들기

In [44]:
import math
def haversine(my_location, target_location):
    '''
    DESCRIPTION: 두 위도경도 사이의 대원거리(great circle distance)를 계산. 위도경도는 각도 단위로 계산.
    '''
    latitude1, longitude1 = my_location
    latitude2, longitude2 = target_location
    lat1, lon1, lat2, lon2 = map(math.radians, [latitude1, longitude1, latitude2, longitude2])
    
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    
    a = math.sin(d_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    meter = 6367 * c * 1000
    return round(meter, 2)

### 3. 거리 정보 저장하기

In [46]:
# 계산된 거리 정보 리스트화 하기
distance_list = []
for r in school_df[['latitude', 'longitude']].values:
    distance_list.append(haversine(my_location, r))

In [47]:
distance_list[:3]

[5277.85, 3441.52, 1879.41]

In [48]:
#기존의 school_df 데이터프레임 데이터에 정보 추가하기
school_df['distance'] = distance_list

In [49]:
school_df

,name,address,latitude,longitude,distance
0,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053,5277.85
1,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971,3441.52
2,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268,1879.41
3,행신고등학교,경기도 고양시 덕양구 행신동 790번지,37.616131,126.842504,35495.06
4,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672,7773.66
5,초지고등학교,경기도 안산시 단원구 광덕1로 13,37.309868,126.810919,2667.32
6,불곡고등학교,경기도 성남시 분당구 금곡로 149번지,37.348080,127.119199,25651.42


In [50]:
#거리 순으로 데이터 정렬하기
near_school_df = school_df.sort_values('distance')
near_school_df.reset_index(drop=True, inplace=True)
near_school_df

,name,address,latitude,longitude,distance
0,양지고등학교,경기도 안산시 단원구 고잔동 713번지,37.309535,126.822268,1879.41
1,초지고등학교,경기도 안산시 단원구 광덕1로 13,37.309868,126.810919,2667.32
2,상록고등학교,경기도 안산시 상록구 본오동 산127-1번지,37.299510,126.874971,3441.52
3,원곡고등학교,경기도 안산시 단원구 원곡동 965-1번지,37.333300,126.798053,5277.85
4,안산고등학교,경기도 안산시 상록구 청룡4길 8,37.360044,126.873672,7773.66
5,불곡고등학교,경기도 성남시 분당구 금곡로 149번지,37.348080,127.119199,25651.42
6,행신고등학교,경기도 고양시 덕양구 행신동 790번지,37.616131,126.842504,35495.06


In [51]:
#가장 가까운 곳 찾기
nearest_latlng = near_school_df.loc[0][['latitude', 'longitude']].values

### 4. 지도로 표시하기 

In [53]:
# 지도 
hanyang_univ_map2 = folium.Map(location=[37.300853, 126.842930], zoom_start=13)
# 식당표시
folium.CircleMarker(nearest_latlng, popup=near_school_df.loc[0]['name'], color='red', fill_color='red', radius=5).add_to(hanyang_univ_map2)
# 내 위치 표시
folium.CircleMarker(my_location, popup=my_location_name,color='blue', fill_color='blue', radius=5).add_to(hanyang_univ_map2)
hanyang_univ_map2